# F-тест: совместная значимость

In [2]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from scipy.stats import f # F-distribution

# product function

Рассмотрим модель `log(output)` на `log(capital), log(labour), log(wage)` (модель с постоянной эластичностью)

Расширим её, добавив квадраты предикторов `log^2(capital), log^2(labour), log^2(wage)`

Уровень значимости $\alpha=5\%$

Неробастные тесты

In [7]:
labour_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Labour.csv')
mod1 = smf.ols(formula='np.log(output)~np.log(capital)+np.log(labour)+np.log(wage)+I(np.log(capital)**2)+I(np.log(labour)**2)+I(np.log(wage)**2)', 
               data=labour_df).fit()
print(mod1.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:         np.log(output)   R-squared:                       0.896
Model:                            OLS   Adj. R-squared:                  0.895
No. Observations:                 569   F-statistic:                     805.2
Covariance Type:            nonrobust   Prob (F-statistic):          3.40e-272
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -3.5683      1.129     -3.161      0.002      -5.786      -1.351
np.log(capital)             0.1404      0.014      9.754      0.000       0.112       0.169
np.log(labour)              0.4716      0.071      6.642      0.000       0.332       0.611
np.log(wage)                0.4762      0.604      0.789      0.430      -0.709       1.662
I(np.log(capital) ** 2)     0.0071      0.004      1.

Значимость квадратов предикторов

- np.log(capital) ** 2 __незначим__
- np.log(labour) ** 2 __значим__
- np.log(wage) ** 2 __незначим__

Далее, np.log(wage) незначим, np.log(wage) ** 2 незначим

Гипотеза: зарплата не влияет на выпуск

Для нашей модели это означает $H_0:\beta_{\log wage}=\beta_{\log wage**2}
=0$

Используем метод `.f_test()`

In [8]:
mod1.f_test('np.log(wage)=I(np.log(wage) ** 2)=0')

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=108.87883200392028, p=1.084532228455783e-40, df_denom=562, df_num=2>

Для этой гипотезы $P=1.85*10^{40}<\alpha=5\%$ => гипотеза отвергается, влияние зарплаты значимо!

Робастные тесты с поправкой на гетероскедастичность

In [12]:
labour_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Labour.csv')
mod2 = smf.ols(formula='np.log(output)~np.log(capital)+np.log(labour)+np.log(wage)+I(np.log(capital)**2)+I(np.log(labour)**2)+I(np.log(wage)**2)', 
               data=labour_df).fit(cov_type='HC3')
print(mod2.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:         np.log(output)   R-squared:                       0.896
Model:                            OLS   Adj. R-squared:                  0.895
No. Observations:                 569   F-statistic:                     489.4
Covariance Type:                  HC3   Prob (F-statistic):          2.00e-219
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -3.5683      1.651     -2.161      0.031      -6.805      -0.332
np.log(capital)             0.1404      0.030      4.720      0.000       0.082       0.199
np.log(labour)              0.4716      0.198      2.387      0.017       0.084       0.859
np.log(wage)                0.4762      0.895      0.532      0.595      -1.278       2.231
I(np.log(capital) ** 2)     0.0071      0.009      0.

Гипотеза: постоянная эластичность по всем предикторам

В рамках молдели $H_0:\beta_{\log capital **2}=\beta_{\log labour **2}=\beta_{\log wage **2}=0$

In [10]:
mod2.f_test('I(np.log(capital) ** 2)=I(np.log(labour) ** 2)=I(np.log(wage) ** 2)=0')

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=2.0641355692913224, p=0.10389780068289836, df_denom=562, df_num=3>

$P=10.4\%>\alpha=5\%$ => гипотеза не овергается, нелинейные эффеты незначимы!